In [3]:
from pyscf import gto, scf, mp, cc

a = 2 # bond length in a cluster
d = 3 # distance between each cluster
unit = 'b' # unit of length
na = 8  # size of a cluster (monomer)
nc = 1 # set as integer multiple of monomers
spin = 0 # spin per monomer
frozen = 0 # frozen orbital per monomer
elmt = 'H'
basis = '6-31g'
atoms = ""
for n in range(nc*na):
    shift = ((n - n % na) // na) * (d-a)
    atoms += f"{elmt} {n*a+shift:.5f} 0.00000 0.00000 \n"

# atoms = '''
# O  -1.551007  -0.114520   0.000000
# H  -1.934259   0.762503   0.000000
# H  -0.599677   0.040712   0.000000
# O   1.350625   0.111469   0.000000
# H   1.680398  -0.373741  -0.758561
# H   1.680398  -0.373741   0.758561
# '''
# basis = 'ccpvdz'

mol = gto.M(atom=atoms, basis=basis, spin=0, verbose=4, max_memory=16000)
mf = scf.RHF(mol).density_fit()
mf.kernel()


nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()

from pyscf import lo
import numpy as np
nocc = np.count_nonzero(mf.mo_occ)
orbocc = mf.mo_coeff[:,nfrozen:nocc]
lo_coeff  = lo.PipekMezey(mol, orbocc).kernel()

frag_lolist = [[i] for i in range(nocc)]

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-37-generic', version='#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Tue Dec 16 19:46:01 2025
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 8
[INPUT] num. electrons = 8
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000

In [4]:
from collections.abc import Iterable
from pyscf.lno import lnoccsd
# from ad_afqmc.lno_afqmc import ulno_afqmc

thresh = 1e-3
run_frg_list = [0]
chol_cut = 1e-5

mlno = lnoccsd.LNOCCSD(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=4)
mlno.lno_thresh = [thresh*10,thresh]
lno_thresh = mlno.lno_thresh
lno_type = ['1h','1h']
lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
print(lno_thresh)
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
eris = None

if run_frg_list is not None:
    frag_lolist = [frag_lolist[i] for i in run_frg_list]

nfrag = len(frag_lolist)
print(f'number of fragment {nfrag}')
print(f'fragment {frag_lolist}')

if lno_pct_occ is None:
    lno_pct_occ = [None, None]
if lno_norb is None:
    lno_norb = [[None,None]] * nfrag
mf = mlno._scf

if eris is None: eris = mlno.ao2mo()

# Loop over fragment
# frag_res = [None] * nfrag
for ifrag, loidx in enumerate(frag_lolist):
    if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
        orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
        lno_param = [
            [
                {
                    'thresh': (
                        lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                        else lno_thresh[i]
                    ),
                    'pct_occ': (
                        lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                        else lno_pct_occ[i]
                    ),
                    'norb': (
                        lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                        else lno_norb[ifrag][i]
                    ),
                } for i in [0, 1]
            ] for s in range(2)
        ]

    else:
        orbloc = lo_coeff[:,loidx]
        lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                        'norb': lno_norb[ifrag][i]} for i in [0,1]]

    lno_coeff, lno_frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)
    # frag_res[ifrag], frag_msg = mlno.impurity_solve(mf, mo_coeff, uocc_loc, eris, frozen=frozen)
        
        # return (emp2,eccsd,ept)
        # run_afqmc(options,nproc)
        # os.system(f'mv afqmc.out lnoafqmc.out{ifrag+1}')


[0.01, 0.001]
number of fragment 1
fragment [[0]]
LO occ proj: 1 active | 0 standby | 3 orthogonal
LO vir proj: 0 active | 0 standby | 12 orthogonal


In [5]:
lno_frozen

array([ 0,  1,  2,  8,  9, 10, 11, 12, 13, 14, 15])

In [6]:
mol = mf.mol
nocc = np.count_nonzero(mf.mo_occ)
actfrag = np.array([i for i in range(mol.nao) if i not in lno_frozen])
frzocc = np.array([i for i in range(nocc) if i in lno_frozen])
actocc = np.array([i for i in range(nocc) if i in actfrag])
actvir = np.array([i for i in range(nocc,mol.nao) if i in actfrag])
nfrzocc = len(frzocc)
nactocc = len(actocc)
nactvir = len(actvir)
nactorb = len(actfrag)

ncas = nactorb
ncore = nfrzocc
nelec = nactocc*2

print(nelec,ncas,ncore)

2 5 3


In [7]:
from ad_afqmc.lno_afqmc.ccsd_pt2 import lno_afqmc

In [8]:
mo_occ = mlno.mo_occ
lno_frozen, maskact = lnoccsd.get_maskact(lno_frozen, mo_occ.size)

mcc = lnoccsd.CCSD(mf, mo_coeff=lno_coeff, frozen=lno_frozen).set(verbose=4)
mcc._s1e = mlno._s1e
mcc._h1e = mlno._h1e
mcc._vhf = mlno._vhf
if mlno.kwargs_imp is not None:
    mcc = mcc.set(**mlno.kwargs_imp)

(emp2,eccsd,ept), lno_t1, lno_t2, elcorr_t1 =\
    lno_afqmc.solve_lno_ccsd(mcc, lno_coeff, uocc_loc, mo_occ, maskact, ccsd_t=True)

prjlo = uocc_loc @ uocc_loc.T.conj()

# (emp2,eccsd,ept), t1, t2, prjlo =\
#     ulno_afqmc.ulno_ccsd(mcc, lno_coeff, uocc_loc, mo_occ, maskact, ccsd_t=True)
# print(emp2,eccsd,ept)

Init t2, MP2 energy = -3.72198811757153  E_corr(MP2) -0.0387441250205987

******** <class 'pyscf.lno.lnoccsd.MODIFIED_CCSD'> ********
CC2 = 0
CCSD nocc = 1, nmo = 5
frozen orbitals [ 0  1  2  8  9 10 11 12 13 14 15]
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 16000 MB (current use 298 MB)
Init E_corr(MODIFIED_CCSD) = -0.0387441250205987
cycle = 1  E_corr(MODIFIED_CCSD) = -0.0575068544614503  dE = -0.0187627294  norm(t1,t2) = 0.108161
cycle = 2  E_corr(MODIFIED_CCSD) = -0.0682069514782629  dE = -0.010700097  norm(t1,t2) = 0.0645512
cycle = 3  E_corr(MODIFIED_CCSD) = -0.0813173122846959  dE = -0.0131103608  norm(t1,t2) = 0.0355813
cycle = 4  E_corr(MODIFIED_CCSD) = -0.0808182453575436  dE = 0.000499066927  norm(t1,t2) = 0.00267293
cycle = 5  E_corr(MODIFIED_CCSD) = -0.0807904425250151  dE = 2.78028325e-05  norm(t1,t2) = 0.000909778
cycle = 6  E_corr(MODIFIED_CCSD) = -0.0809329081259319  dE

In [1]:
options = {'n_eql': 5,
        'n_prop_steps': 50,
        'n_ene_blocks': 1,
        'n_sr_blocks': 10,
        'n_blocks': 10,
        'n_walkers': 1,
        'seed': 98,
        'walker_type': 'rhf',
        'trial': 'ccsd_pt2',
        'dt':0.005,
        'free_projection':False,
        'ad_mode':None,
        'use_gpu': False,
        'max_error': 1e-4
        }

# from mpi4py import MPI
# if not MPI.Is_finalized():
#     MPI.Finalize()

nelec, ncas = lno_afqmc.prep_afqmc(mf,lno_coeff,lno_t1,lno_t2,lno_frozen,prjlo,
                         options,chol_cut=chol_cut)

print(nelec, ncas)

NameError: name 'lno_afqmc' is not defined

In [3]:
from ad_afqmc import config
from ad_afqmc.lno_afqmc.ccsd_pt2 import lno_afqmc
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

ham_data, prop, trial, wave_data, sampler, options, _ = (
    lno_afqmc._prep_afqmc())

from jax import numpy as jnp
import numpy as np
mo_coeff = jnp.array(np.eye(trial.norb))
wave_data["mo_coeff"] = mo_coeff[:, :trial.nelec[0]]

# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Using GPU.
# System: Linux
# Node Name: sharmagroup-rn
# Release: 6.14.0-37-generic
# Version: #37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2
# Machine: x86_64
# Processor: x86_64
# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Using GPU.
# System: Linux
# Node Name: sharmagroup-rn
# Release: 6.14.0-37-generic
# Version: #37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2
# Machine: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 5
# nelec: 6
#
# n_eql: 3
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 5
# n_blocks: 100
# n_walkers: 10
# seed: 99
# walker_type: rhf
# trial: ccsd_pt2
# dt: 0.005
# free_projection: False
# use_gpu: True
# max_error: 0.001
# n_exp_terms: 6
# ene0: 0.0
# n_batch: 1
#


In [4]:
import time
from jax import numpy as jnp
from jax import random
init_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### initialize propagation
seed = options["seed"]
init_walkers = None
# dm_up = jnp.array(wave_data["mo_coeff"][0] @ wave_data["mo_coeff"][0].T.conj())
# dm_dn = jnp.array(wave_data["mo_coeff"][1] @ wave_data["mo_coeff"][1].T.conj())
# trial_rdm1 = [dm_up, dm_dn]
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = trial._build_measurement_intermediates(ham_data, wave_data)
ham_data = prop._build_propagation_intermediates(ham_data, trial, wave_data)

prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
print(prop_data["e_estimate"])

-148.9632014280337


In [6]:
walker = prop_data['walkers'][0]
walker_bar = wave_data['exp_t1'] @ walker
t2eorb, t2orb, e12bar = trial._t2eorb_bar_tc_old(walker_bar, ham_data, wave_data)
print(t2eorb, t2orb, e12bar)
t2eorb2, t2orb2 = trial._t2e_orb_ad(walker_bar, ham_data, wave_data)
e12bar2 = trial._calc_energy_bar(walker_bar, ham_data, wave_data)
print(t2eorb2, t2orb2, e12bar2)

(-0.11844149114682584+0j) 0j (-13.75384291786923+0j)
(-0.11844149114682581+0j) 0j (-13.75384291786923+0j)


In [40]:
import numpy as np
walker_r = np.random.randn(*walker.shape)
# walker_bar = wave_data['exp_t1'] @ walker_r
t2eorb, t2orb, e12bar = trial._t2eorb_bar_tc_old(walker_r, ham_data, wave_data)
print(t2eorb, t2orb, e12bar)
t2eorb2, t2orb2 = trial._t2e_orb_ad(walker_r, ham_data, wave_data)
e12bar2 = trial._calc_energy_bar(walker_r, ham_data, wave_data)
print(t2eorb2, t2orb2, e12bar2)

21.49961232163743 -1.6917026357065097 -12.038338967342352
21.499612321637404 -1.691702635706509 -12.038338967342348


In [103]:
import opt_einsum as oe
# einsum = jnp.einsum
# einsum = oe.contract

def _t2_orb(walker, wave_data: dict):
    '''
    <HF|t2|walker>_i 
    = t_iajb <HF|i+ j+ a b|walker>/<HF|walker> * <HF|walker>
    = t_iajb (G_ia G_jb-G_ib G_ja) * <HF|walker>
    prj onto orbital i
    '''
    nocc = walker.shape[1]
    t2 = wave_data["t2_prj"]
    gf = (walker.dot(jnp.linalg.inv(walker[:nocc, :]))).T
    o0 = jnp.linalg.det(walker[: nocc, :]) ** 2
    # o1 = oe.contract("ia,ia->", t1, gf[:, nocc:], backend="jax")
    o2 = 2 * oe.contract("iajb,ia,jb->", t2, gf[:, nocc:], gf[:, nocc:], backend="jax") \
        - oe.contract("iajb,ib,ja->", t2, gf[:, nocc:], gf[:, nocc:], backend="jax")
    olp = o2 * o0
    return olp

def _t2_orb_exp1(x, h1_mod, walker,
                wave_data):
    '''
    <HF|t2_i exp(x*h1_mod)|walker>
    '''
    walker_1x = walker + x*h1_mod.dot(walker)
    olp = _t2_orb(walker_1x, wave_data)
    # o0 = jnp.linalg.det(walker[: walker.shape[1], :]) ** 2
    return olp


def _t2_orb_exp2(x, chol_i, walker, wave_data):
    '''
    <HF|t2_i exp(x*h2_mod)|walker>
    '''
    walker_2x = (
            walker
            + x * chol_i.dot(walker)
            + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker))
        )
    olp = _t2_orb(walker_2x, wave_data)
    # o0 = jnp.linalg.det(walker[: walker.shape[1], :]) ** 2
    return olp

from jax import vmap, lax, jvp
def _d2_exp2_i(chol_i,walker, wave_data):
    x = 0.0
    f = lambda a: _t2_orb_exp2(a,chol_i,walker,wave_data)
    _, d2f = jvp(lambda x: jvp(f, [x], [1.0])[1], [x], [1.0])
    return d2f

In [110]:
import opt_einsum as oe
from jax import vmap, lax, jvp

def _t2eorb_ad(trial, walker, ham_data, wave_data):
    '''
    <HF|t2_i(h1mod+h2mod)|walker>/<HF|walker>
    '''

    norb = trial.norb
    chol = ham_data["chol"].reshape(-1, norb, norb)
    h1_mod = ham_data['h1_mod']
    h1 = ham_data['h1'][0]
    # h0_E0 = ham_data["h0"]-ham_data["E0"]

    nocc = walker.shape[1]
    o0 = jnp.linalg.det(walker[: nocc, :]) ** 2

    x = 0.0
    # one body
    f1 = lambda a: _t2_orb_exp1(a,h1_mod,walker,wave_data)
    tolp, d_overlap_mod = jvp(f1, [x], [1.0])

    f1 = lambda a: _t2_orb_exp1(a,h1,walker,wave_data)
    _, d_overlap_h1 = jvp(f1, [x], [1.0])

    # two body
    def scanned_fun(carry, c):
        walker, wave_data = carry
        return carry, _d2_exp2_i(c,walker,wave_data)

    _, d2_olp2_i = lax.scan(scanned_fun, (walker, wave_data), chol)
    d_2_overlap = jnp.sum(d2_olp2_i)/2

    # <hf|(t1+t2)_i (h1+h2)|walker>/<hf|walker>
    e1mod = d_overlap_mod/o0
    e1 = d_overlap_h1/o0
    e2mod = d_2_overlap/o0
    e2 = e1mod - e1 + e2mod
    # teorb = (d_overlap + d_2_overlap) / o0
    # teorb = e1 + e2
    torb = tolp/o0 # <t2_i>

    return jnp.real(e1mod), jnp.real(e2mod), jnp.real(e1), jnp.real(e2), jnp.real(torb)

def _t2eorb_tc(trial, walker, ham_data, wave_data):
    nocc, norb = trial.nelec[0], trial.norb
    t2 = wave_data["t2_prj"]
    green = (walker.dot(jnp.linalg.inv(walker[:nocc, :]))).T
    green_occ = green[:, nocc:]
    greenp = jnp.vstack((green_occ, -jnp.eye(norb - nocc)))

    chol = ham_data["chol"].reshape(-1, norb, norb)
    rot_chol = chol[:, :nocc, :]
    h1 = ham_data["h1"][0]
    hg = oe.contract("pi,pi->", h1[:nocc, :], green, backend="jax")

    # 1 body energy
    # ref
    e1_0 = 2 * hg

    # double excitations
    t2g_c_1 = oe.contract("iajb,ia->jb", t2, green_occ, backend="jax")
    t2g_c_2 = oe.contract("iajb,jb->ia", t2, green_occ, backend="jax")
    t2g_e_1 = oe.contract("iajb,ib->ja", t2, green_occ, backend="jax")
    t2g_e_2 = oe.contract("iajb,ja->ib", t2, green_occ, backend="jax")
    t2_green_c_1 = oe.contract("pb,jb,jq->pq", greenp, t2g_c_1, green, backend="jax") # t_iajb G_ia G_jq Gp_pb (-)
    t2_green_c_2 = oe.contract("pa,ia,iq->pq", greenp, t2g_c_2, green, backend="jax") # t_iajb G_jb G_iq Gp_pa (-)
    t2_green_e_1 = oe.contract("pa,ja,jq->pq", greenp, t2g_e_1, green, backend="jax") # t_iajb G_ib G_jq Gp_pa (+)
    t2_green_e_2 = oe.contract("pb,ib,iq->pq", greenp, t2g_e_2, green, backend="jax") # t_iajb G_ja G_iq Gp_pb (+)
    t2g_c = t2g_c_1 + t2g_c_2
    t2g_e = t2g_e_1 + t2g_e_2
    t2_green_c = t2_green_c_1 + t2_green_c_2
    t2_green_e = t2_green_e_1 + t2_green_e_2
    t2_green = t2_green_c - t2_green_e * 0.5
    t2g = t2g_c - t2g_e * 0.5
    gt2g = oe.contract("ia,ia->", t2g, green_occ, backend="jax")
    e1_2_1 = 2 * hg * gt2g
    e1_2_2 = -2 * oe.contract("pq,pq->", h1, t2_green, backend="jax")
    e1_2 = e1_2_1 + e1_2_2

    # two body energy
    # ref
    lg_c = oe.contract("gip,ip->g", rot_chol, green, backend="jax")
    lg_e = oe.contract("gip,jp->gij", rot_chol, green, backend="jax")
    e2_0_1 = 2 * lg_c @ lg_c
    e2_0_2 = -jnp.sum(vmap(lambda x: x * x.T)(lg_e))
    e2_0 = e2_0_1 + e2_0_2

    # double excitations
    e2_2_1 = e2_0 * gt2g
    # e2_mod = 0.5 * oe.contract("gpr,grq,pq->", chol, chol, t2_green, backend="jax")
    lt2g = oe.contract("gpr,pr->g", chol, t2_green, backend="jax")
    e2_2_2_1 = -lt2g @ lg_c # t_iajb |G_ia G_jq Gp_pb L_pr| G_qs L_qs

    def scanned_fun(carry, x):
        chol_i, rot_chol_i = x
        gl_i = oe.contract("ir,qr->iq", green, chol_i, backend="jax")
        lt2_green_i = oe.contract("ir,qr->iq", rot_chol_i, t2_green, backend="jax")
        # t_iajb |G_ia G_js Gp_pb| G_qr L_pr L_qs
        carry[0] += 0.5 * oe.contract("iq,iq->", gl_i, lt2_green_i, backend="jax")
        # t_iajb G_ir G_js Gp_pa Gp_qb L_pr L_qs type
        glgp_i = oe.contract("ir,rb->ib", gl_i, greenp, backend="jax")
        l2t2_c = oe.contract("iajb,ia,jb->", t2, glgp_i, glgp_i, backend="jax")
        # l2t2_c_2 = oe.contract("iajb,jb,ia->", t2, glgp_i, glgp_i, backend="jax")
        l2t2_e = oe.contract("iajb,ib,ja->", t2, glgp_i, glgp_i, backend="jax")
        # l2t2_e_2 = oe.contract("iajb,ja,ib->", t2, glgp_i, glgp_i, backend="jax")
        # l2t2_c = l2t2_c_1 + l2t2_c_2
        # l2t2_e = l2t2_e_1 + l2t2_e_2
        carry[1] += 2*l2t2_c - l2t2_e
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(scanned_fun, [0.0, 0.0], (chol, rot_chol))
    e2_2_2 = 4 * (e2_2_2_1 + e2_2_2_2)

    e2_2 = e2_2_1 + e2_2_2 + e2_2_3

    e0 = e1_0 + e2_0 # <psi|(h1+h2)|phi>/<psi|phi>
    te = e1_2 + e2_2 # <psi|t2(h1+h2)|phi>/<psi|phi>

    t = gt2g # <psi|t2|phi>/<psi|phi>

    return jnp.real(e1_2), jnp.real(e2_2), jnp.real(t), jnp.real(e0)


In [ ]:
def _t2eorb_tc_new(trial, walker, ham_data, wave_data):
    nocc, norb = trial.nelec[0], trial.norb
    t2 = wave_data["t2_prj"]
    green = (walker.dot(jnp.linalg.inv(walker[:nocc, :]))).T
    green_occ = green[:, nocc:]
    greenp = jnp.vstack((green_occ, -jnp.eye(norb - nocc)))

    chol = ham_data["chol"].reshape(-1, norb, norb)
    rot_chol = chol[:, :nocc, :]
    h1 = ham_data["h1"][0]
    hg = oe.contract("pi,pi->", h1[:nocc, :], green, backend="jax")

    # 1 body energy
    # ref
    e1_0 = 2 * hg

    # double excitations
    # t_iajb =! t_jbia since the i axis is projected onto LNO !!!
    t2g_c_1 = oe.contract("iajb,ia->jb", t2, green_occ, backend="jax")
    t2g_c_2 = oe.contract("iajb,jb->ia", t2, green_occ, backend="jax")
    t2g_e_1 = oe.contract("iajb,ib->ja", t2, green_occ, backend="jax")
    t2g_e_2 = oe.contract("iajb,ja->ib", t2, green_occ, backend="jax")
    t2_green_c_1 = oe.contract("pb,jb,jq->pq", greenp, t2g_c_1, green, backend="jax") # t_iajb G_ia G_jq Gp_pb (-)
    t2_green_c_2 = oe.contract("pa,ia,iq->pq", greenp, t2g_c_2, green, backend="jax") # t_iajb G_jb G_iq Gp_pa (-)
    t2_green_e_1 = oe.contract("pa,ja,jq->pq", greenp, t2g_e_1, green, backend="jax") # t_iajb G_ib G_jq Gp_pa (+)
    t2_green_e_2 = oe.contract("pb,ib,iq->pq", greenp, t2g_e_2, green, backend="jax") # t_iajb G_ja G_iq Gp_pb (+)
    t2g_c = t2g_c_1 + t2g_c_2
    t2g_e = t2g_e_1 + t2g_e_2
    t2_green_c = t2_green_c_1 + t2_green_c_2
    t2_green_e = t2_green_e_1 + t2_green_e_2
    t2_green = t2_green_c - t2_green_e * 0.5
    t2g = t2g_c - t2g_e * 0.5
    gt2g = oe.contract("ia,ia->", t2g, green_occ, backend="jax")
    e1_2_1 = 2 * hg * gt2g
    e1_2_2 = -2 * oe.contract("pq,pq->", h1, t2_green, backend="jax")
    e1_2 = e1_2_1 + e1_2_2

    # two body
    # e2_2_1 = e2_0 * gt2g
    # e2_mod = 0.5 * oe.contract("gpr,grq,pq->", chol, chol, t2_green, backend="jax")
    # lt2g = oe.contract("gpr,pr->g", chol, t2_green, backend="jax")
    # e2_2_2_1 = -lt2g @ lg_c # t_iajb |G_ia G_jq Gp_pb L_pr| G_qs L_qs
    def scanned_fun(carry, x):
        chol_i, rot_chol_i = x
        gl_i = oe.contract("ir,qr->iq", green, chol_i, backend="jax")
        gl_c_i = oe.contract("ii->",gl_i[:,:nocc], backend="jax")
        e2_0_c_i = gl_c_i**2
        e2_0_e_i = oe.contract("ij,ji->",gl_i[:,:nocc], gl_i[:,:nocc], backend="jax")
        carry[0] += 2 * e2_0_c_i - e2_0_e_i
        lt2g_i = oe.contract("pr,pr->", chol_i, t2_green, backend="jax")
        carry[1] += -lt2g_i * gl_c_i
        lt2_green_i = oe.contract("ir,qr->iq", rot_chol_i, t2_green, backend="jax")
        # t_iajb |G_ia G_js Gp_pb| G_qr L_pr L_qs
        carry[2] += 0.5 * oe.contract("iq,iq->", gl_i, lt2_green_i, backend="jax")
        # t_iajb G_ir G_js Gp_pa Gp_qb L_pr L_qs type
        glgp_i = oe.contract("ir,rb->ib", gl_i, greenp, backend="jax")
        l2t2_c = oe.contract("iajb,ia,jb->", t2, glgp_i, glgp_i, backend="jax")
        # l2t2_c_2 = oe.contract("iajb,jb,ia->", t2, glgp_i, glgp_i, backend="jax")
        l2t2_e = oe.contract("iajb,ib,ja->", t2, glgp_i, glgp_i, backend="jax")
        # l2t2_e_2 = oe.contract("iajb,ja,ib->", t2, glgp_i, glgp_i, backend="jax")
        # l2t2_c = l2t2_c_1 + l2t2_c_2
        # l2t2_e = l2t2_e_1 + l2t2_e_2
        carry[3] += 2*l2t2_c - l2t2_e
        return carry, 0.0

    [e2_0, e2_2_2_1, e2_2_2_2, e2_2_3], _ \
        = lax.scan(scanned_fun, [0.0, 0.0, 0.0, 0.0], (chol, rot_chol))
    e2_2_1 = e2_0 * gt2g
    e2_2_2 = 4 * (e2_2_2_1 + e2_2_2_2)
    e2_2 = e2_2_1 + e2_2_2 + e2_2_3

    e0 = e1_0 + e2_0 # <psi|(h1+h2)|phi>/<psi|phi>
    te = e1_2 + e2_2 # <psi|t2(h1+h2)|phi>/<psi|phi>

    t = gt2g # <psi|t2|phi>/<psi|phi>

    return jnp.real(e1_2), jnp.real(e2_2), jnp.real(t), jnp.real(e0)


In [27]:
chol = ham_data['chol'].reshape(-1,trial.norb,trial.norb)
v0 = 0.5 * jnp.einsum("gpr,grq->pq", chol, chol, optimize="optimal")
h1_mod = ham_data['h1'][0] - v0
ham_data['h1_mod'] = h1_mod
# print(h1_mod)

In [100]:
t2 = wave_data['t2']
t2_prj = jnp.einsum('iajb,ik->kajb',t2,wave_data['prjlo'])
wave_data['t2_prj'] = t2_prj
# wave_data['t2_prj'] = t2

In [121]:
walker = prop_data['walkers'][0]
e1_tc1, e2_tc1, t_tc1, e0_tc = _t2eorb_tc(trial, walker, ham_data, wave_data)
print(e1_tc1 + e2_tc1, e1_tc1, e2_tc1, t_tc1)
print(e0_tc)
e1_tc1, e2_tc1, t_tc1, e0_tc = _t2eorb_tc_new(trial, walker, ham_data, wave_data)
print(e1_tc1 + e2_tc1, e1_tc1, e2_tc1, t_tc1)
print(e0_tc)
e1mod_ad, e2mod_ad, e1_ad, e2_ad, t_ad = _t2eorb_ad(trial, walker, ham_data, wave_data)
print(e1mod_ad + e2mod_ad, e1_ad + e2_ad, e1_ad, e2_ad, t_ad)
e0_ad = jnp.real(trial._calc_energy_restricted(walker, ham_data, wave_data))
print(e0_ad - ham_data['h0'])

-0.005020096905385424 -0.0 -0.005020096905385424 0.0
-13.753854418210919
-0.005020096905385424 -0.0 -0.005020096905385424 0.0
-13.753854418210922
-0.005020096905385424 -0.005020096905385424 0.0 -0.005020096905385424 0.0
-13.75385441821092


In [124]:
import numpy as np
walker_r = np.random.randn(*walker.shape)
e1_tc1, e2_tc1, t_tc1, e0_tc = _t2eorb_tc(trial, walker_r, ham_data, wave_data)
print(e1_tc1 + e2_tc1, e1_tc1, e2_tc1, t_tc1, e0_tc)
e1_tc2, e2_tc2, t_tc2, e0_tc2 = _t2eorb_tc_new(trial, walker_r, ham_data, wave_data)
print(e1_tc2 + e2_tc2, e1_tc2, e2_tc2, t_tc2, e0_tc2)
e1mod_ad, e2mod_ad, e1_ad, e2_ad, t_ad = _t2eorb_ad(trial, walker_r, ham_data, wave_data)
e0_ad = jnp.real(trial._calc_energy_restricted(walker_r, ham_data, wave_data))
print(e1mod_ad + e2mod_ad, e1_ad, e2_ad, t_ad, e0_ad - ham_data['h0'])
# e1_fd, e2_fd, t_fd = _calc_energy_fd(trial, walker_r, ham_data, wave_data)
# print(e1_fd, e2_fd, t_fd)

8.130450494502085 62.83009457525839 -54.69964408075631 -2.288659091609131 90.31580606857878
8.130450494502085 62.83009457525839 -54.69964408075631 -2.288659091609131 90.31580606857881
8.130450494502199 62.83009457525841 -54.69964408075621 -2.288659091609131 90.31580606857878
